In [1]:
!pip install -r requirements.txt
!mkdir dataset
!mkdir dataset\InventoryImages\

     ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
     ----- ---------------------------------- 0.2/1.4 MB 11.5 MB/s eta 0:00:01
     ------------ --------------------------- 0.5/1.4 MB 5.6 MB/s eta 0:00:01
     ------------- -------------------------- 0.5/1.4 MB 2.9 MB/s eta 0:00:01
     ---------------------------------- ----- 1.2/1.4 MB 5.2 MB/s eta 0:00:01
     ---------------------------------------- 1.4/1.4 MB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 0.0/7.7 MB ? eta -:--:--
     --- ------------------------------------ 0.6/7.7 MB 20.5 MB/s eta 0:00:01
     ------- -------------------------------- 1.4/7.7 MB 17.2 MB/s eta 0:00:01
     ---------- ----------------------------- 2.1/7.7 MB 14.8 MB/s eta 0:00:01
     --------------- ------------------------ 2.9/7.7 MB 14.2 MB/s eta 0:00:01
     --------------- ------------------------ 2.9

In [3]:
# After downloading and labeling images
!labelme2yolo --json_dir ./dataset/LabelFiles/ --val_size 0.2 --test_size 0.2 --output_format bbox

[2024-10-11T12:17:17Z INFO  labelme2yolo] Starting the conversion process...
[2024-10-11T12:17:17Z WARN  labelme2yolo] Directory "./dataset/LabelFiles/YOLODataset/labels" already exists. Deleting and recreating it.
[2024-10-11T12:17:17Z WARN  labelme2yolo] Directory "./dataset/LabelFiles/YOLODataset/images" already exists. Deleting and recreating it.
[2024-10-11T12:17:17Z INFO  labelme2yolo] Read and parsed 10 files.
[2024-10-11T12:17:17Z INFO  labelme2yolo] Creating dataset.yaml file...
[2024-10-11T12:17:17Z INFO  labelme2yolo] Conversion process completed successfully.


In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO('yolov8n.pt')  # load a pretrained model (recommended for training)

# Train the model
results = model.train(data=r".\dataset\LabelFiles\YOLODataset\dataset.yaml", epochs=1000, imgsz=640, batch=12, device='cuda')

# Export the model
model.export(format="torchscript")

# モデルから物体検知

In [1]:
from ultralytics import YOLO
# Load a model
model = YOLO(r'./runs/detect/Recall_0.999/weights/best.pt')

# Validate the model
metrics = model.val()  # no arguments needed, dataset and settings remembered
metrics.box.map    # map50-95
metrics.box.map50  # map50
metrics.box.map75  # map75
metrics.box.maps   # a list contains map50-95 of each category
print(metrics)

# Predict the model
result = model.predict('dataset/InventoryImages/Tushonka_5.png', save=True, conf=0.25)
for r in result:
    labels=r.names
    for box in r.boxes:
        print(f"ラベル: {labels[int(box.cls[0])]}, 位置: {box.xyxy}")

Ultralytics YOLOv8.2.27  Python-3.11.7 torch-2.3.0+cpu CPU (11th Gen Intel Core(TM) i3-1115G4 3.00GHz)
Model summary (fused): 168 layers, 3006038 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning \\?\C:\Users\spyk4\git\TarkovInventoryAalyzer\dataset\LabelFiles\YOLODataset\labels\val.cache... 2 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2/2 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:02<00:00,  2.25s/it]


                   all          2         17      0.219     0.0556     0.0849     0.0157
Can of beef stew (Small)          2          8          0          0     0.0632     0.0161
Can of beef stew (Large)          1          9      0.438      0.111      0.107     0.0154
Speed: 0.0ms preprocess, 208.7ms inference, 0.0ms loss, 824.9ms postprocess per image
Results saved to runs\detect\val4
ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000001D23C268A10>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.0

# データ問い合わせ

In [2]:
import requests
items=[]
for r in result:
    labels=r.names
    print(labels)
    for box in r.boxes:
        query = """
        {
  items(names: "%s") {
    id
    name
    buyFor {
      price
      source
      currency
    }
    sellFor {
      currency
      price
      vendor {
        name
      }
    }
  }
}
        """
        label_name = labels[int(box.cls[0])]
        response = requests.post(
            'https://api.tarkov.dev/graphql',
            json={'query': query % label_name}
        )
        item_data: dict = response.json()
        if response.status_code == 200:
            items.append(item_data)
            print(item_data)
            data = response.json()
        else:
            print(f"GraphQL request failed with status code {response.status_code}")


{0: 'Can of beef stew (Small)', 1: 'Can of beef stew (Large)'}
{'data': {'items': [{'id': '57347d7224597744596b4e72', 'name': 'Can of beef stew (Small)', 'buyFor': [{'price': 16624, 'source': 'fleaMarket', 'currency': 'RUB'}], 'sellFor': [{'currency': 'RUB', 'price': 8978, 'vendor': {'name': 'Therapist'}}, {'currency': 'RUB', 'price': 5700, 'vendor': {'name': 'Fence'}}, {'currency': 'RUB', 'price': 8551, 'vendor': {'name': 'Jaeger'}}, {'currency': 'RUB', 'price': 17000, 'vendor': {'name': 'Flea Market'}}]}]}}
{'data': {'items': [{'id': '57347d7224597744596b4e72', 'name': 'Can of beef stew (Small)', 'buyFor': [{'price': 16624, 'source': 'fleaMarket', 'currency': 'RUB'}], 'sellFor': [{'currency': 'RUB', 'price': 8978, 'vendor': {'name': 'Therapist'}}, {'currency': 'RUB', 'price': 5700, 'vendor': {'name': 'Fence'}}, {'currency': 'RUB', 'price': 8551, 'vendor': {'name': 'Jaeger'}}, {'currency': 'RUB', 'price': 17000, 'vendor': {'name': 'Flea Market'}}]}]}}
{'data': {'items': [{'id': '57347

# 最良の販売方法

In [8]:
import json
for item in items:
    best_sell_option = None
    best_price = 0
    for sell_option in item['data']['items'][0]['sellFor']:
        if sell_option['price'] > best_price:
            best_price = sell_option['price']
            best_sell_option = sell_option
    if best_sell_option:
        print(f"最良の販売方法: {best_sell_option['vendor']['name']}, 価格: {best_price} {best_sell_option['currency']}")
    else:
        print("取引が不可能または販売価格が0を下回っています。")

最良の販売方法: Flea Market, 価格: 17000 RUB
最良の販売方法: Flea Market, 価格: 17000 RUB
最良の販売方法: Flea Market, 価格: 17000 RUB
最良の販売方法: Flea Market, 価格: 17000 RUB
最良の販売方法: Flea Market, 価格: 17000 RUB
最良の販売方法: Flea Market, 価格: 17777 RUB
最良の販売方法: Flea Market, 価格: 17777 RUB
最良の販売方法: Flea Market, 価格: 17000 RUB
最良の販売方法: Flea Market, 価格: 17000 RUB
最良の販売方法: Flea Market, 価格: 17777 RUB
最良の販売方法: Flea Market, 価格: 17777 RUB
最良の販売方法: Flea Market, 価格: 17777 RUB
最良の販売方法: Flea Market, 価格: 17777 RUB
最良の販売方法: Flea Market, 価格: 17000 RUB
最良の販売方法: Flea Market, 価格: 17777 RUB
最良の販売方法: Flea Market, 価格: 17777 RUB
最良の販売方法: Flea Market, 価格: 17777 RUB


# どのタスクでどの程度使うか

In [12]:
i=0
for item in items:
    i+=1
    with open('./taskItems.json', 'r', encoding='utf-8') as f:
        objective_data = json.load(f)
        for task in objective_data['data']['tasks']:
            for objective in task['objectives']:
                if 'items' in objective:
                    for task_item in objective["items"]:
                        detected_item_name = item['data']['items'][0]['name']
                        if detected_item_name ==task_item["name"]:
                            print(f"{i}. {detected_item_name}x{objective['count']} is needed for {task['name']}.")

1. Can of beef stew (Small)x15 is needed for General Wares.
1. Can of beef stew (Small)x15 is needed for General Wares.
2. Can of beef stew (Small)x15 is needed for General Wares.
2. Can of beef stew (Small)x15 is needed for General Wares.
3. Can of beef stew (Small)x15 is needed for General Wares.
3. Can of beef stew (Small)x15 is needed for General Wares.
4. Can of beef stew (Small)x15 is needed for General Wares.
4. Can of beef stew (Small)x15 is needed for General Wares.
5. Can of beef stew (Small)x15 is needed for General Wares.
5. Can of beef stew (Small)x15 is needed for General Wares.
6. Can of beef stew (Large)x2 is needed for Acquaintance.
7. Can of beef stew (Large)x2 is needed for Acquaintance.
8. Can of beef stew (Small)x15 is needed for General Wares.
8. Can of beef stew (Small)x15 is needed for General Wares.
9. Can of beef stew (Small)x15 is needed for General Wares.
9. Can of beef stew (Small)x15 is needed for General Wares.
10. Can of beef stew (Large)x2 is needed for

# どのハイドアウトでどの程度使うか

In [24]:
i=0
for item in items:
    i+=1
    with open('./hideoutItems.json', 'r', encoding='utf-8') as f:
        objective_data = json.load(f)
        for station in objective_data['data']['hideoutStations']:
            station_name=station["name"]
            for level in station['levels']:
                level_num=level["level"]
                for requestedItem in level["itemRequirements"]:
                    item_count=requestedItem["count"]
                    detected_item_name = item['data']['items'][0]['name']
                    if detected_item_name == requestedItem['item']['name']:
                        print(f"{i}. {detected_item_name}x{item_count} is needed for {station_name} level {level_num}.")